In [1]:
# Dependencies
import pandas as pd
import requests
import psycopg2
import numpy as np
from sqlalchemy import create_engine, table

In [9]:
# Importing recipes from spoonacular
api_key = "a21364208c9d43719315d65c34faa384"  ### will need to grab from config file
# Using url for "Get Recipe Information" JSONs: https://spoonacular.com/food-api/docs#Get-Recipe-Information
# url = "https://api.spoonacular.com/recipes/10/information?includeNutrition=false"

id_df = pd.read_csv('spoonacular_recipes.csv')
id = id_df['1']

# id = range(10)

# Initializing lists for url and query_url
# url = []
# query_url = [None]*len(id)

# Loop through a set of recipes up to a pre-determined amount
# for i in range(0, len(id)):
#     url.append(f"https://api.spoonacular.com/recipes/{id[i]}/information?includeNutrition=false")
#     query_url[i] = f'{url[i]}&apiKey={api_key}'
id_df

,Unnamed: 0,Spoonacular Recipe ID,Recipe,Recipe Image URL,Recipe Image Type,Recipe Calories,Recipe Protein Content (grams),Recipe Fat Content (grams),Recipe Carbs Content (grams)
0,0,649131,Kung Po Chicken,https://spoonacular.com/recipeImages/649131-31...,jpg,478,33g,34g,13g
1,1,656516,Poached Sole With Blood Orange Beurre Blanc,https://spoonacular.com/recipeImages/656516-31...,jpg,311,29g,19g,4g
2,2,640843,Crispy Southern Fried Chicken,https://spoonacular.com/recipeImages/640843-31...,jpg,613,36g,35g,36g
3,3,648467,Japanese Cucumber Salad,https://spoonacular.com/recipeImages/648467-31...,jpg,26,1g,1g,4g
4,4,643561,Fresh Peach With Goat Cheese Side Salad,https://spoonacular.com/recipeImages/643561-31...,jpg,179,7g,7g,26g
...,...,...,...,...,...,...,...,...,...
2698,2698,661510,Steak Salad With Roasted Potatoes and Fennel,https://spoonacular.com/recipeImages/661510-31...,jpg,873,43g,46g,76g
2699,2699,663448,Tiramisu Infused With Amaretto (Eggless!),https://spoonacular.com/recipeImages/663448-31...,jpg,199,4g,12g,16g
2700,2700,646499,Healthy Orange Chicken,https://spoonacular.com/recipeImages/646499-31...,jpg,440,52g,8g,34g
2701,2701,643789,Fried Salmon Cakes,https://spoonacular.com/recipeImages/643789-31...,jpg,696,27g,31g,79g


In [3]:
from pprint import pprint
# Initializing lists for results from query
recipe_id = []
recipe_name = []
servings = []
diets = []
dairyFree = []
vegan = []
vegetarian = []
ketogenic = []
images = []
source_url = []
ingredients = []
individual_ingre=[]

# Looping through recipe JSONs
count = 0
ingred_list = []

for i in range(0, len(id)):
    
    try:
        random_list = []
        response = requests.get(query_url[i]).json()
    
        recipe_id.append(response['id'])
        recipe_name.append(response['title'])
        servings.append(response['servings'])
        diets.append(response['diets'])
        dairyFree.append(response['dairyFree'])
        vegan.append(response['vegan'])
        vegetarian.append(response['vegetarian'])
        images.append(response['image'])
        source_url.append(response['sourceUrl'])
        ingredients.append(response['extendedIngredients'])
        
        for ingre in ingredients:
            for i in range(len(ingre)):
                random_list.append(ingre[i]["name"])
        individual_ingre.append(random_list)
    except:
        print(f'Recipe {id[i]} was not found.')

Recipe 663641 was not found.
Recipe 664025 was not found.
Recipe 655847 was not found.
Recipe 639606 was not found.
Recipe 664780 was not found.
Recipe 640259 was not found.
Recipe 648524 was not found.
Recipe 632897 was not found.
Recipe 632901 was not found.
Recipe 641093 was not found.
Recipe 649292 was not found.
Recipe 665680 was not found.
Recipe 641104 was not found.
Recipe 665694 was not found.
Recipe 641122 was not found.
Recipe 641202 was not found.
Recipe 641671 was not found.
Recipe 633537 was not found.
Recipe 641908 was not found.
Recipe 634660 was not found.
Recipe 634751 was not found.
Recipe 659638 was not found.
Recipe 660286 was not found.
Recipe 635712 was not found.
Recipe 660406 was not found.
Recipe 636581 was not found.
Recipe 636600 was not found.
Recipe 661291 was not found.
Recipe 637222 was not found.
Recipe 653775 was not found.
Recipe 646491 was not found.
Recipe 663121 was not found.
Recipe 663227 was not found.


In [35]:
# Develop dataframe from queried recipes
recipe_df = pd.DataFrame({'ID': recipe_id,
                          'Recipe Name': recipe_name,
                          'Servings': servings,
                          'Diets': diets,
                          'Dairy Free': dairyFree,
                          'Vegan': vegan,
                          'Vegetarian': vegetarian
## It looks like some recipes don't have an Image URL, source URL, or Ingredients list.
## So these lists come up short compared to the above lists. 
## Commented out for the time being. Might need to add an "if" statement during the appending in the above cell.
#                           'Image URL': images, 
#                           'Source URL': source_url,
#                           'Ingredients': individual_ingre
                        })
                        
recipe_forsql_df = recipe_df.rename(columns = {'Recipe Name':'Recipe_Name',
                                               'Dairy Free':'Dairy_Free',
                                               'Image URL':'Image_URL',
                                               'Source URL':'Source_URL'
                                              })
# Display recipe_df
recipe_df

,ID,Recipe Name,Servings,Diets,Dairy Free,Vegan,Vegetarian
0,1515523,Instant Pot BBQ Chicken Thighs,4,"[gluten free, dairy free]",True,False,False
1,663559,Tomato and lentil soup,4,"[gluten free, dairy free, lacto ovo vegetarian...",True,True,True
2,638996,Chocolate Dipped Coconut Macaroons,9,[gluten free],False,False,False
3,647197,Honey Chipotle Pork Ribs,16,"[gluten free, dairy free]",True,False,False
4,655392,Pear and Apple Crumble,8,"[dairy free, lacto ovo vegetarian, vegan]",True,True,True
...,...,...,...,...,...,...,...
1941,655330,Peanut Butter Rice Krispie Treats,15,[],False,False,False
1942,647142,Homestlye Chicken Noodle Casserole,8,[],False,False,False
1943,663535,Tokwa Baboy (Braised Tofu With Pork),4,[dairy free],True,False,False
1944,655347,Peanut Butter-Oatmeal Chocolate Chip Cookies,10,[lacto ovo vegetarian],False,False,True


In [6]:
rds_connection_string = 'postgres:postgres@localhost:5432/Project_2'
engine = create_engine(f'postgresql://{rds_connection_string}')

recipe_df.to_sql('recipe_df', con=engine, index=False, if_exists='replace')

In [36]:
len(recipe_id)

1946

In [37]:
len(images)

1922

0       1515523
1        663559
2        638996
3        647197
4        655392
         ...   
1950     655330
1951     647142
1952     663535
1953     655347
1954     663545
Name: 0, Length: 1955, dtype: int64